![CH3-ADS.png](.\Media\CH3-ADS.png)

# <span style="color:rgb(204, 85, 0);">Get Locked Pages in Memory information</span>

<span style="font-size:14px;">This notebook will test to see if LPIM has been enabled.&nbsp;&nbsp;</span> [Server memory configuration options - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/database-engine/configure-windows/server-memory-server-configuration-options?view=sql-server-ver15)  and [Enable the Lock Pages in Memory Option (Windows) - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/database-engine/configure-windows/enable-the-lock-pages-in-memory-option-windows?view=sql-server-ver15) and [Configuration: Lock Pages in Memory (LPIM) - Brent Ozar Unlimited®](https://www.brentozar.com/training/fundamentals-database-administration/lock-pages-memory-lpim-6m/) are some help resources to review.

### <span style="color:rgb(0, 204, 153);">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
SET NOCOUNT ON;
SET ANSI_WARNINGS ON;
SET QUOTED_IDENTIFIER ON;
SET DATEFORMAT mdy;

DECLARE @sqlcmd NVARCHAR(max), @params NVARCHAR(600);
DECLARE @sqlmajorver int, @sqlminorver int, @sqlbuild int;
DECLARE @ErrorMessage NVARCHAR(4000);
DECLARE @masterpid int;
DECLARE @permstbl TABLE ([name] sysname);
DECLARE @lpim bit, @lognumber int, @logcount int;
DECLARE @arch smallint, @osver VARCHAR(5), @ostype VARCHAR(10), @StartDate DATETIME;

SELECT @masterpid = principal_id FROM master.sys.database_principals (NOLOCK) WHERE sid = SUSER_SID()

INSERT INTO @permstbl
SELECT a.name
FROM master.sys.all_objects a (NOLOCK) INNER JOIN master.sys.database_permissions b (NOLOCK) ON a.[OBJECT_ID] = b.major_id
WHERE a.type IN ('P', 'X') AND b.grantee_principal_id <>0
AND b.grantee_principal_id <>2
AND b.grantee_principal_id = @masterpid;

SELECT @sqlmajorver = CONVERT(int, (@@microsoftversion / 0x1000000) & 0xff);
SELECT @sqlminorver = CONVERT(int, (@@microsoftversion / 0x10000) & 0xff);
SELECT @sqlbuild = CONVERT(int, @@microsoftversion & 0xffff);

IF (@sqlmajorver >= 11 AND @sqlmajorver < 14) OR (@sqlmajorver = 10 AND @sqlminorver = 50 AND @sqlbuild >= 2500)
BEGIN
	SET @sqlcmd = N'SELECT @ostypeOUT = ''Windows'', @osverOUT = CASE WHEN windows_release IN (''6.3'',''10.0'') AND (@@VERSION LIKE ''%Build 10586%'' OR @@VERSION LIKE ''%Build 14393%'') THEN ''10.0'' ELSE windows_release END, @archOUT = CASE WHEN @@VERSION LIKE ''%<X64>%'' THEN 64 WHEN @@VERSION LIKE ''%<IA64>%'' THEN 128 ELSE 32 END FROM sys.dm_os_windows_info (NOLOCK)';
	SET @params = N'@osverOUT VARCHAR(5) OUTPUT, @ostypeOUT VARCHAR(10) OUTPUT, @archOUT smallint OUTPUT';
	EXECUTE sp_executesql @sqlcmd, @params, @osverOUT=@osver OUTPUT, @ostypeOUT=@ostype OUTPUT, @archOUT=@arch OUTPUT;
END
ELSE IF @sqlmajorver >= 14
BEGIN
	SET @sqlcmd = N'SELECT @ostypeOUT = host_platform, @osverOUT = CASE WHEN host_platform = ''Windows'' AND host_release IN (''6.3'',''10.0'') THEN ''10.0'' ELSE host_release END, @archOUT = CASE WHEN @@VERSION LIKE ''%<X64>%'' THEN 64 ELSE 32 END FROM sys.dm_os_host_info (NOLOCK)';
	SET @params = N'@osverOUT VARCHAR(5) OUTPUT, @ostypeOUT VARCHAR(10) OUTPUT, @archOUT smallint OUTPUT';
	EXECUTE sp_executesql @sqlcmd, @params, @osverOUT=@osver OUTPUT, @ostypeOUT=@ostype OUTPUT, @archOUT=@arch OUTPUT;
END;
ELSE
BEGIN
	BEGIN TRY
		DECLARE @str VARCHAR(500), @str2 VARCHAR(500), @str3 VARCHAR(500)
		DECLARE @sysinfo TABLE (id int, 
			[Name] NVARCHAR(256), 
			Internal_Value bigint, 
			Character_Value NVARCHAR(256));
			
		INSERT INTO @sysinfo
		EXEC xp_msver;
		
		SELECT @osver = LEFT(Character_Value, CHARINDEX(' ', Character_Value)-1) -- 5.2 is WS2003; 6.0 is WS2008; 6.1 is WS2008R2; 6.2 is WS2012, 6.3 is WS2012R2, 6.3 (14396) is WS2016
		FROM @sysinfo
		WHERE [Name] LIKE 'WindowsVersion%';
		
		SELECT @arch = CASE WHEN RTRIM(Character_Value) LIKE '%x64%' OR RTRIM(Character_Value) LIKE '%AMD64%' THEN 64
			WHEN RTRIM(Character_Value) LIKE '%x86%' OR RTRIM(Character_Value) LIKE '%32%' THEN 32
			WHEN RTRIM(Character_Value) LIKE '%IA64%' THEN 128 END
		FROM @sysinfo
		WHERE [Name] LIKE 'Platform%';

		SET @ostype = 'Windows'
	END TRY
	BEGIN CATCH
		SELECT ERROR_NUMBER() AS ErrorNumber, ERROR_MESSAGE() AS ErrorMessage;
		SELECT @ErrorMessage = 'Windows Version and Architecture subsection - Error raised in TRY block. ' + ERROR_MESSAGE()
		RAISERROR (@ErrorMessage, 16, 1);
	END CATCH
END;

IF ((@sqlmajorver = 13 AND @sqlbuild >= 4000) OR @sqlmajorver > 13)
BEGIN
	SET @sqlcmd = N'SELECT @lpimOUT = CASE WHEN sql_memory_model = 2 THEN 1 ELSE 0 END FROM sys.dm_os_sys_info (NOLOCK)';
	SET @params = N'@lpimOUT bit OUTPUT';
	EXECUTE sp_executesql @sqlcmd, @params, @lpimOUT=@lpim OUTPUT;
END

IF ((@sqlmajorver = 13 AND @sqlbuild < 4000) OR (@sqlmajorver >= 10 AND @sqlmajorver < 13))
BEGIN
	SET @sqlcmd = N'SELECT @lpimOUT = CASE WHEN locked_page_allocations_kb > 0 THEN 1 ELSE 0 END FROM sys.dm_os_process_memory (NOLOCK)'
	SET @params = N'@lpimOUT bit OUTPUT';
	EXECUTE sp_executesql @sqlcmd, @params, @lpimOUT=@lpim OUTPUT
END

IF @sqlmajorver = 9
BEGIN
	IF ISNULL(IS_SRVROLEMEMBER(N'sysadmin'), 0) = 1 -- Is sysadmin
		OR ISNULL(IS_SRVROLEMEMBER(N'securityadmin'), 0) = 1 -- Is securityadmin
		OR ((SELECT COUNT([name]) FROM @permstbl WHERE [name] = 'sp_readerrorlog') > 0
			AND (SELECT COUNT([name]) FROM @permstbl WHERE [name] = 'xp_readerrorlog') > 0
			AND (SELECT COUNT([name]) FROM @permstbl WHERE [name] = 'xp_enumerrorlogs') > 0)
	BEGIN
		IF EXISTS (SELECT [object_id] FROM tempdb.sys.objects (NOLOCK) WHERE [object_id] = OBJECT_ID('tempdb.dbo.#lpimdbcc'))
		DROP TABLE #lpimdbcc;
		IF NOT EXISTS (SELECT [object_id] FROM tempdb.sys.objects (NOLOCK) WHERE [object_id] = OBJECT_ID('tempdb.dbo.#lpimdbcc'))
		CREATE TABLE #lpimdbcc (logdate DATETIME, spid VARCHAR(50), logmsg VARCHAR(4000))

		IF EXISTS (SELECT [object_id] FROM tempdb.sys.objects (NOLOCK) WHERE [object_id] = OBJECT_ID('tempdb.dbo.#lpimavail_logs'))
		DROP TABLE #lpimavail_logs;
		IF NOT EXISTS (SELECT [object_id] FROM tempdb.sys.objects (NOLOCK) WHERE [object_id] = OBJECT_ID('tempdb.dbo.#lpimavail_logs'))
		CREATE TABLE #lpimavail_logs (lognum int, logdate DATETIME, logsize int) 

		-- Get the number of available logs 
		INSERT INTO #lpimavail_logs 
		EXEC xp_enumerrorlogs 

		-- Get Start date
		SET @sqlcmd = N'SELECT @StartDateOUT = login_time FROM master..sysprocesses (NOLOCK) WHERE spid = 1';
		
        SET @params = N'@StartDateOUT DATETIME OUTPUT';

		EXECUTE sp_executesql @sqlcmd, @params, @StartDateOUT=@StartDate OUTPUT;
		
		SELECT @lognumber = MIN(lognum) FROM #lpimavail_logs WHERE DATEADD(dd, DATEDIFF(dd, 0, logdate), 0) >= DATEADD(dd, DATEDIFF(dd, 0, @StartDate), 0)

		SELECT @logcount = ISNULL(MAX(lognum),@lognumber) FROM #lpimavail_logs WHERE DATEADD(dd, DATEDIFF(dd, 0, logdate), 0) >= DATEADD(dd, DATEDIFF(dd, 0, @StartDate), 0)

		IF @lognumber IS NULL
		BEGIN
			SELECT @ErrorMessage = 'WARNING: Could not retrieve information about Locked pages usage in SQL Server 2005'
			RAISERROR (@ErrorMessage, 16, 1);
		END
		ELSE
		WHILE @lognumber < @logcount 
		BEGIN
			-- Cycle through sql error logs (Cannot use Large Page Extensions:  lock memory privilege was not granted)
			SELECT @sqlcmd = 'EXEC master..sp_readerrorlog ' + CONVERT(VARCHAR(3),@lognumber) + ', 1, ''Using locked pages for buffer pool'''
			BEGIN TRY
				INSERT INTO #lpimdbcc (logdate, spid, logmsg) 
				EXECUTE (@sqlcmd);
			END TRY
			BEGIN CATCH
				SELECT ERROR_NUMBER() AS ErrorNumber, ERROR_MESSAGE() AS ErrorMessage;
				SELECT @ErrorMessage = 'Errorlog based subsection - Error raised in TRY block 1. ' + ERROR_MESSAGE()
				RAISERROR (@ErrorMessage, 16, 1);
			END CATCH
			-- Next log 
			--SET @lognumber = @lognumber + 1 
			SELECT @lognumber = MIN(lognum) FROM #lpimavail_logs WHERE lognum > @lognumber
		END 

		IF (SELECT COUNT(*) FROM #lpimdbcc) > 0
		BEGIN
			SET @lpim = 1
		END
		ELSE IF (SELECT COUNT(*) FROM #lpimdbcc) = 0 AND @lognumber IS NOT NULL
		BEGIN
			SET @lpim = 0
		END;
		
		DROP TABLE #lpimavail_logs;
		DROP TABLE #lpimdbcc;
	END
	ELSE
	BEGIN
		RAISERROR('WARNING: Only a sysadmin or securityadmin can run the "Locked_pages" check. Bypassing check', 16, 1, N'permissions')
		RAISERROR('WARNING: If not sysadmin or securityadmin, then user must be a granted EXECUTE permissions on the following sprocs to run checks: xp_enumerrorlogs and sp_readerrorlog. Bypassing check', 16, 1, N'extended_sprocs')
		--RETURN
	END;
END

IF @lpim = 0 AND CONVERT(DECIMAL(3,1), @osver) < 6.0 AND @arch = 64
BEGIN
	SELECT 'Memory_checks' AS [Category], 'Locked_pages' AS [Check], 'WARNING: Locked pages are not in use by SQL Server. In a WS2003 x64 architecture it is recommended to enable LPIM' AS [Deviation]
END
ELSE IF @lpim = 1 AND CONVERT(DECIMAL(3,1), @osver) < 6.0 AND @arch = 64
BEGIN
	SELECT 'Memory_checks' AS [Category], 'Locked_pages' AS [Check], 'INFORMATION: Locked pages are being used by SQL Server. This is recommended in a WS2003 x64 architecture' AS [Deviation]
END
ELSE IF @lpim = 1 AND CONVERT(DECIMAL(3,1), @osver) >= 6.0 AND @arch = 64
BEGIN
	SELECT 'Memory_checks' AS [Category], 'Locked_pages' AS [Check], 'INFORMATION: Locked pages are being used by SQL Server. This is recommended in WS2008 or above only when there are signs of paging' AS [Deviation]
END
ELSE IF @lpim IS NULL
BEGIN
	SELECT 'Memory_checks' AS [Category], 'Locked_pages' AS [Check], 'Could not retrieve information' AS [Deviation]
END
ELSE
BEGIN
	SELECT 'Memory_checks' AS [Category], 'Locked_pages' AS [Check], 'Not in use' AS [Deviation]
END;